## Spark (Apache)
- Spark SQL
- Spark Streaming
- MLib (machine learning)
- GraphX (graph)

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
spark

In [3]:
# Read a csv file and set the headers
df = (spark.read.options(header="true").csv("../data_lake/landing/ratings.csv"))

df.show()

+------------+-------+---------------+-------+
|       brand|  model|absorption_rate|comfort|
+------------+-------+---------------+-------+
|Diapers-R-Us|6months|              2|      3|
|     Nappy-k|2months|              3|      4|
|     Pampers|3months|              4|      4|
|     Huggies|newborn|              3|      5|
+------------+-------+---------------+-------+



In [4]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Define the schema
schema = StructType([
  StructField("brand", StringType(), nullable=False),
  StructField("model", StringType(), nullable=False),
  StructField("absorption_rate", IntegerType(), nullable=True),
  StructField("comfort", IntegerType(), nullable=True)
])

better_df = (spark
             .read
             .options(header="true")
             # Pass the predefined schema to the Reader
             .schema(schema)
             .csv("../data_lake/landing/ratings.csv"))
print(better_df.dtypes)

[('brand', 'string'), ('model', 'string'), ('absorption_rate', 'int'), ('comfort', 'int')]


### Cleaning Data
- Incorrectdatatypes
- Invalidrows
- Incompleterows
- Badlychosenplaceholders

In [5]:
#Removing malformed data
ratings = (spark
           .read
           .options(header=True, mode="DROPMALFORMED")
           .csv("../data_lake/landing/ratings_with_invalid_rows.csv"))
ratings.show()

+------------+-------+---------------+-------+
|       brand|  model|absorption_rate|comfort|
+------------+-------+---------------+-------+
|Diapers-R-Us|6months|              2|      3|
|     Nappy-k|2months|              3|      4|
|     Pampers|3months|              4|      4|
|     Huggies|newborn|              3|      5|
+------------+-------+---------------+-------+



In [6]:
#Fill NAN data
print("BEFORE")
ratings.show()

print("AFTER")
# Replace nulls with arbitrary value on column subset
ratings = ratings.fillna(4, subset=["comfort"])
ratings.show()

BEFORE
+------------+-------+---------------+-------+
|       brand|  model|absorption_rate|comfort|
+------------+-------+---------------+-------+
|Diapers-R-Us|6months|              2|      3|
|     Nappy-k|2months|              3|      4|
|     Pampers|3months|              4|      4|
|     Huggies|newborn|              3|      5|
+------------+-------+---------------+-------+

AFTER
+------------+-------+---------------+-------+
|       brand|  model|absorption_rate|comfort|
+------------+-------+---------------+-------+
|Diapers-R-Us|6months|              2|      3|
|     Nappy-k|2months|              3|      4|
|     Pampers|3months|              4|      4|
|     Huggies|newborn|              3|      5|
+------------+-------+---------------+-------+



In [7]:
from pyspark.sql.functions import col, when

# Add/relabel the column
categorized_ratings = ratings.withColumn(
    "comfort",
    # Express the condition in terms of column operations
    when(col("comfort") > 3, "sufficient").otherwise("insufficient"))

categorized_ratings.show()

+------------+-------+---------------+------------+
|       brand|  model|absorption_rate|     comfort|
+------------+-------+---------------+------------+
|Diapers-R-Us|6months|              2|insufficient|
|     Nappy-k|2months|              3|  sufficient|
|     Pampers|3months|              4|  sufficient|
|     Huggies|newborn|              3|  sufficient|
+------------+-------+---------------+------------+



In [8]:
from pyspark.sql.functions import col

# Select the columns and rename the "absorption_rate" column
result = ratings.select([col("brand"),
                       col("model"),
                       col("absorption_rate").alias("absorbency")])

# Show only unique values
result.distinct().show()

+------------+-------+----------+
|       brand|  model|absorbency|
+------------+-------+----------+
|     Pampers|3months|         4|
|Diapers-R-Us|6months|         2|
|     Huggies|newborn|         3|
|     Pampers|    2mo|      null|
|     Nappy-k|2months|         3|
+------------+-------+----------+



In [9]:
from pyspark.sql.functions import col, avg, stddev_samp, max as sfmax

aggregated = (purchased
              # Group rows by 'Country'
              .groupBy(col('Country'))
              .agg(
                # Calculate the average salary per group and rename
                avg('Salary').alias('average_salary'),
                # Calculate the standard deviation per group
                stddev_samp('Salary'),
                # Retain the highest salary per group and rename
                sfmax('Salary').alias('highest_salary')
              )
             )

aggregated.show()

NameError: name 'purchased' is not defined

### Production pipeline files

## Creating in-memory DataFrames


In [10]:
from datetime import date
from pyspark.sql import Row

Record = Row("country", "utm_campaign", "airtime_in_minutes", "start_date", "end_date")

# Create a tuple of records
data = (
  Record("USA", "DiapersFirst", 28, date(2017, 1, 20), date(2017, 1, 27)),
  Record("Germany", "WindelKind", 31, date(2017, 1, 25), None),
  Record("India", "CloseToCloth", 32, date(2017, 1, 25), date(2017, 2, 2))
)

# Create a DataFrame from these records
frame = spark.createDataFrame(data)
frame.show()

Py4JJavaError: An error occurred while calling o75.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 10.0 failed 1 times, most recent failure: Lost task 0.0 in stage 10.0 (TID 9) (TRZPG100430.intra.trizy.com.br executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\Users\luis.ribeiro\Anaconda3\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 485, in main
RuntimeError: Python in worker has different version 3.7 than that in driver 3.8, PySpark cannot run with different minor versions. Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:555)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:713)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:695)
	at org.apa